# Body parts

- Loss function: ArcFace
- SwimB
- closed set
- Segmentation
  - Full body - Combine all body parts
  - Weighted body parts - Combine all body parts with Weights

In [1]:
import os
from pathlib import Path

In [2]:
from src.dataset import download_dataset
from src.utils import get_device
from src.pipeline import TurtleReIdPipeline

In [3]:
# --- Pipeline parameters ---
device = get_device()
EMBEDDING_SIZE = 512


# --- Transform parameters ---
IMG_SIZE = 224


# --- Datasets parameters ---
DATA_DIR = './data/seaturtleid2022-subset'
train_csv_path = os.path.join(DATA_DIR, "metadata_closed_set_splits_train.csv")
eval_csv_path = os.path.join(DATA_DIR, "metadata_closed_set_splits_valid.csv")
test_csv_path = os.path.join(DATA_DIR, "metadata_closed_set_splits_test.csv")
datasets_paths = {
    'train_csv_path': train_csv_path,
    'eval_csv_path': eval_csv_path,
    'test_csv_path': test_csv_path,
}

paths = download_dataset()
img_dir=paths['images_path']

annotations_path= f"{DATA_DIR}/annotations.json"


# --- Training parameters ---
BATCH_SIZE = 10
LEARNING_RATE = 1e-4
EPOCHS = 10

model_dir = './models'
Path(model_dir).mkdir(parents=True, exist_ok=True)


train_configs = {
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'epochs': EPOCHS,
    # 'model_save_path': model_save_path,
    'k': 1
}

Dataset downloaded and extracted to: /Users/nhut/.cache/kagglehub/datasets/wildlifedatasets/seaturtleid2022/versions/4


### Whole turtle body

In [ ]:
turtle_pipeline = TurtleReIdPipeline(device, EMBEDDING_SIZE)
turtle_pipeline.set_transforms(IMG_SIZE)
turtle_pipeline.set_datasets(datasets_paths, img_dir, annotations_path)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [ ]:
# --- Training Loop on All Parts ---
turtle_configs = train_configs.copy()
model_save_path = f'{model_dir}/04_parts_combination_closed_arcface_swimb.pth'
turtle_configs['model_save_path'] = model_save_path
turtle_configs['target_part'] = 'turtle'  # all parts (head, flipper, body)
turtle_pipeline.train(turtle_configs)

Epoch 1/10:   0%|          | 0/21 [00:00<?, ?it/s]

### Whole Turtle body with Trainable weights

In [4]:
weights_pipeline = TurtleReIdPipeline(
    device, EMBEDDING_SIZE, use_weighted_parts=True)
weights_pipeline.set_transforms(IMG_SIZE)
weights_pipeline.set_datasets(datasets_paths, img_dir, annotations_path)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [ ]:
# --- Training Loop on WEIGHTED Parts ---
weights_configs = train_configs.copy()
model_save_path = f'{model_dir}/04_weighted_parts_combination_closed_arcface_swimb.pth'
weights_configs['model_save_path'] = model_save_path
weights_configs['target_part'] = 'weighted_parts'  # trainable weights parts
weights_pipeline.train(weights_configs)

Epoch 1/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 1/10, Loss: 16.9480, Eval Accuracy: 13.41%
Saved best model.


Epoch 2/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 2/10, Loss: 15.8703, Eval Accuracy: 23.17%
Saved best model.


Epoch 3/10:   0%|          | 0/21 [00:00<?, ?it/s]

...

Epoch 8/10, Loss: 12.2449, Eval Accuracy: 45.12%


Epoch 9/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 9/10, Loss: 10.9753, Eval Accuracy: 42.68%


Epoch 10/10:   0%|          | 0/21 [00:00<?, ?it/s]

Epoch 10/10, Loss: 10.3774, Eval Accuracy: 42.68%
Finished Training. Best Test Accuracy: 46.34%
